In [133]:
import numpy as np

from dandi import dandiapi
from elephant.current_source_density import generate_lfp, estimate_csd
from elephant.current_source_density_src.utility_functions import gauss_1d_dipole, small_source_2D
from neo import AnalogSignal
from pynwb import NWBHDF5IO

In [117]:
# dandiset_id = "000021"
# dandi_filepath = "sub-703279277/sub-703279277_ses-719161530.nwb"
dandiset_id = "000336"
dandi_filepath = "sub_1171903433/sub_1171903433sess_1181330601/sub_1171903433+sess_1181330601+probeA_ecephys.nwb"

authenticate = False
dandi_api_key = ""

In [118]:
if authenticate:
    client = dandiapi.DandiAPIClient(token=dandi_api_key)
else:
    client = dandiapi.DandiAPIClient()
my_dandiset = client.get_dandiset(dandiset_id)

filename = dandi_filepath.split("/")[-1]
filepath = f"./{filename}"
file = my_dandiset.get_asset_by_path(dandi_filepath)

Error 401 while sending GET request to https://api.dandiarchive.org/api/dandisets/000336/: {"detail":"Authentication credentials were not provided."}


HTTPError: Error 401 while sending GET request to https://api.dandiarchive.org/api/dandisets/000336/: {"detail":"Authentication credentials were not provided."}

In [ ]:
# # this may take awhile, especially if the file to download is large
# file.download(filepath)

# print(f"Downloaded file to {filepath}")

In [120]:
io = NWBHDF5IO("../../../data/illusion/sub_1171903433+sess_1181330601+probeA_ecephys.nwb", mode="r", load_namespaces=True)
nwb = io.read()

C:\Users\carter.peene\AppData\Local\Programs\Python\Python39\lib\site-packages\hdmf\spec\namespace.py:531: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.5.0 because version 1.5.1 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
C:\Users\carter.peene\AppData\Local\Programs\Python\Python39\lib\site-packages\hdmf\spec\namespace.py:531: UserWarning: Ignoring cached namespace 'core' version 2.4.0 because version 2.5.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
C:\Users\carter.peene\AppData\Local\Programs\Python\Python39\lib\site-packages\hdmf\spec\namespace.py:531: UserWarning: Ignoring cached namespace 'hdmf-experimental' version 0.1.0 because version 0.2.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."


### Elephant

In [107]:
nwb.electrodes[:50]

,x,y,z,imp,location,filtering,group,group_name,probe_vertical_position,probe_horizontal_position,probe_id,local_index,valid_data
id,,,,,,,,,,,,,
850261194,7648.0,3645.0,3645.0,NaN,PO,AP band: 500 Hz high-pass; LFP band: 1000 Hz l...,probeC abc.EcephysElectrodeGroup at 0x22317315...,probeC,20,43,810755801,0,True
850261196,7651.0,3636.0,3636.0,NaN,PO,AP band: 500 Hz high-pass; LFP band: 1000 Hz l...,probeC abc.EcephysElectrodeGroup at 0x22317315...,probeC,20,11,810755801,1,True
850261202,7660.0,3610.0,3610.0,NaN,PO,AP band: 500 Hz high-pass; LFP band: 1000 Hz l...,probeC abc.EcephysElectrodeGroup at 0x22317315...,probeC,60,43,810755801,4,True
850261206,7665.0,3592.0,3592.0,NaN,PO,AP band: 500 Hz high-pass; LFP band: 1000 Hz l...,probeC abc.EcephysElectrodeGroup at 0x22317315...,probeC,80,59,810755801,6,True
850261212,7674.0,3566.0,3566.0,NaN,PO,AP band: 500 Hz high-pass; LFP band: 1000 Hz l...,probeC abc.EcephysElectrodeGroup at 0x22317315...,probeC,100,11,810755801,9,True
850261218,7685.0,3531.0,3531.0,NaN,PO,AP band: 500 Hz high-pass; LFP band: 1000 Hz l...,probeC abc.EcephysElectrodeGroup at 0x22317315...,probeC,140,43,810755801,12,True
850261222,7690.0,3513.0,3513.0,NaN,PO,AP band: 500 Hz high-pass; LFP band: 1000 Hz l...,probeC abc.EcephysElectrodeGroup at 0x22317315...,probeC,160,59,810755801,14,True
850261224,7693.0,3505.0,3505.0,NaN,PO,AP band: 500 Hz high-pass; LFP band: 1000 Hz l...,probeC abc.EcephysElectrodeGroup at 0x22317315...,probeC,160,27,810755801,15,True
850261230,7701.0,3478.0,3478.0,NaN,PoT,AP band: 500 Hz high-pass; LFP band: 1000 Hz l...,probeC abc.EcephysElectrodeGroup at 0x22317315...,probeC,200,59,810755801,18,True


In [150]:
lfp = nwb.acquisition["probe_0_lfp_data"]
lfp

probe_0_lfp_data pynwb.ecephys.ElectricalSeries at 0x2231736399472
Fields:
  comments: no comments
  conversion: 1.0
  data: <HDF5 dataset "data": shape (11252278, 89), type "<f4">
  description: no description
  electrodes: electrodes <class 'hdmf.common.table.DynamicTableRegion'>
  interval: 1
  offset: 0.0
  resolution: -1.0
  timestamps: <HDF5 dataset "timestamps": shape (11252278,), type "<f8">
  timestamps_unit: seconds
  unit: volts

In [159]:
import quantities as pq
coords = np.array(nwb.electrodes.x)
hz = len(lfp.data) / lfp.timestamps[-1]

neo_lfp = AnalogSignal(lfp.data, units="V", sampling_rate = hz*pq.Hz, coordinates = coords * pq.mm)
neo_lfp

AnalogSignal with 89 channels of length 11252278; units V; datatype float32 
annotations: {'coordinates': array([8501., 8490., 8479., 8465., 8454., 8443., 8432., 8420., 8409.,
         8395., 8384., 8373., 8362., 8351., 8340., 8326., 8315., 8304.,
         8293., 8282., 8271., 8257., 8246., 8235., 8224., 8213., 8202.,
         8189., 8178., 8167., 8156., 8145., 8132., 8121., 8108., 8095.,
         8081., 8068., 8058., 8044., 8031., 8017., 8006., 7992., 7978.,
         7963., 7949., 7937., 7922., 7908., 7894., 7883., 7869., 7856.,
         7844., 7832., 7822., 7811., 7801., 7792., 7784., 7778., 7771.,
         7764., 7757., 7751., 7743., 7735., 7727., 7718., 7710., 7701.,
         7691., 7681., 7674., 7664., 7655., 7648., 7642., 7635., 7629.,
         7621., 7615., 7609., 7602., 7595., 7587., 7577., 7568.]) * mm}
sampling rate: 1249.3370604199158 Hz
time: 0.0 s to 9006.599064801605 s

In [160]:
fake_lfp = generate_lfp(small_source_2D, xs, ys)
fake_lfp

AnalogSignal with 89 channels of length 1; units mV; datatype float64 
annotations: {'coordinates': array([[8501., 3565.],
         [8490., 3526.],
         [8479., 3488.],
         [8465., 3441.],
         [8454., 3402.],
         [8443., 3365.],
         [8432., 3327.],
         [8420., 3289.],
         [8409., 3251.],
         [8395., 3205.],
         [8384., 3168.],
         [8373., 3130.],
         [8362., 3094.],
         [8351., 3057.],
         [8340., 3020.],
         [8326., 2975.],
         [8315., 2939.],
         [8304., 2903.],
         [8293., 2867.],
         [8282., 2831.],
         [8271., 2796.],
         [8257., 2752.],
         [8246., 2717.],
         [8235., 2682.],
         [8224., 2647.],
         [8213., 2613.],
         [8202., 2578.],
         [8189., 2535.],
         [8178., 2501.],
         [8167., 2466.],
         [8156., 2432.],
         [8145., 2398.],
         [8132., 2354.],
         [8121., 2320.],
         [8108., 2276.],
         [8095., 2231.],
  

In [161]:
csd = estimate_csd(neo_lfp, method="KCSD2D")

TypeError: len() of unsized object

In [99]:
csd_a = np.array(csd)